In [17]:
import re
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [9]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt

myclient = pymongo.MongoClient("mongodb://localhost:27017")
db = myclient["robin"]
collection = db["testPythonDB"]

# Retrieve all Documents
df_products = pd.DataFrame(list(collection.find({})))

keep_columns = [
    "brandName",
    "name"
]
df_products = df_products[keep_columns]

df_products.head(5)

,brandName,name
0,BURGER KING,menu familiar
1,SAMSUNG,Televisor Samsung 65''
2,NO IDENTIFICADA,Crema hidratante facial
3,NO IDENTIFICADA,cepillo de dientes electrico
4,BURGER KING,menu familiar


In [28]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return str(only_ascii)[2:]

stemmer = PorterStemmer()
nltk.download('stopwords')
words = stopwords.words("spanish")
df_products['cleaned'] = df_products['name'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", remove_accents(str(x))).split() if i not in words]).lower())

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
vectorizer = TfidfVectorizer(min_df= 3, sublinear_tf=True, norm='l2', ngram_range=(1, 2))
final_features = vectorizer.fit_transform(df_products['cleaned']).toarray()
final_features.shape

(2699, 2113)